In [1]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import re
from Webscraping import scrape_tiki


TIKI_URL = 'https://tiki.vn'

conn = sqlite3.connect('tiki.db')
cur = conn.cursor()

def get_url(url):
    try:
        response = requests.get(url).text
        soup = BeautifulSoup(response, 'html.parser')
        return soup
    except Exception as err:
        print('ERROR BY REQUEST:', err)


def create_categories_table():
    query = """
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_id INTEGER, 
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)
        
def create_products_table():
    query = """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            category VARCHAR(255),
            name VARCHAR(255),
            final_price INTEGER,
            regular_price INTEGER,
            discount_percentage INTEGER,
            installment VARCHAR(255),
            cross_border VARCHAR(255),
            sponsor VARCHAR(255),
            reviews INTEGER,
            rating INTEGER,
            rating_by_stars INTEGER,
            url TEXT,
            image_url TEXT
        )"""
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)

class Category:
    def __init__(self, name, url, parent_id=None, cat_id=None):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id

    def __repr__(self):
        return f"ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent: {self.parent_id}"

    def save_into_db(self):
        query = """
            INSERT INTO categories (name, url, parent_id)
            VALUES (?, ?, ?);
        """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            self.cat_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)

            
class Product:
    def __init__(self, category, name, final_price, regular_price, 
                 discount_percentage, installment, cross_border, 
                 sponsor, reviews, rating, rating_by_stars, url, image_url, product_id=None):
        self.product_id = product_id
        self.category = category
        self.name = name
        self.final_price = final_price
        self.regular_price = regular_price
        self.discount_percentage = discount_percentage
        self.installment = installment
        self.cross_border = cross_border
        self.sponsor = sponsor
        self.reviews = reviews
        self.rating = rating
        self.rating_by_stars = rating_by_stars
        self.url = url
        self.image_url = image_url
        
    def __repr__(self):
        return f"ID: {self.product_id}, Category: {self.category}, Name: {self.name}, Final Price: {self.final_price}, Regular Price: {self.regular_price}"
    
    def save_into_db(self):
        query = """
            INSERT INTO products (category, name, final_price,
                                    regular_price, discount_percentage,
                                    installment, cross_border, sponsor,
                                    reviews, rating, rating_by_stars, url, image_url)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """
        val = (self.category, self.name, self.final_price,
                  self.regular_price, self.discount_percentage,
                  self.installment, self.cross_border, self.sponsor,
                  self.reviews, self.rating, self.rating_by_stars, self.url, self.image_url)
        try:
            cur.execute(query, val)
            self.product_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)
            
            
def get_main_categories(save_db=False):
    soup = get_url(TIKI_URL)

    result = []
    for a in soup.find_all('a', {'class': 'MenuItem__MenuLink-sc-181aa19-1 fKvTQu'}):
        name = a.find('span', {'class': 'text'}).text
        url = a['href']
        main_cat = Category(name, url)

        if save_db:
            main_cat.save_into_db()
        result.append(main_cat)
    return result

def get_sub_categories(parent_category, save_db=False):
    parent_url = parent_category.url
    result = []

    try:
        soup = get_url(parent_url)
        div_containers = soup.find_all('div', {'class':'list-group-item is-child'})
        for div in div_containers:
            name = div.a.text

            # replace more than 2 spaces with one space
            name = re.sub('\s{2,}', ' ', name)

            sub_url = TIKI_URL + div.a['href']
            cat = Category(name, sub_url, parent_category.cat_id)
            if save_db:
                cat.save_into_db()
            result.append(cat)
    except Exception as err:
        print('ERROR BY GET SUB CATEGORIES:', err)
    return result

def get_all_categories(categories):
    if len(categories) == 0:
        return
    for cat in categories:
        sub_categories = get_sub_categories(cat, save_db=True)
        print(sub_categories)
        get_all_categories(sub_categories)

def get_product_details(save_db=True):
    urls = cur.execute("""SELECT url
        FROM categories
        WHERE id
        NOT IN (
            SELECT DISTINCT parent_id 
            FROM categories
            WHERE parent_id NOTNULL)
        AND parent_id NOTNULL""").fetchall()
    
    if save_db:
        for url in urls:
             for each in (scrape_tiki(url[0])):
                product = Product(each['Category'], each['Name'], each['Final_price'], each['Regular_price'],
                                     each['Discount_percent'], each['Installment'], each['Cross_border'],
                                     each['Sponsor'], each['Reviews'], each['Rating'], each['Rating_by_stars'],
                                     each['Url'], each['Image_url'])
                product.save_into_db()

# cur.execute('DROP TABLE products;')
# cur.execute('DROP TABLE categories;')
# conn.commit()
create_categories_table()
create_products_table()
main_categories = get_main_categories(save_db=True)
get_all_categories(main_categories)
get_product_details()


2
[ID: 17, Name:  Máy tính bảng (40)
, URL: https://tiki.vn/may-tinh-bang/c1794?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1, ID: 18, Name:  Máy đọc sách (21)
, URL: https://tiki.vn/may-doc-sach/c28856?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1, ID: 19, Name:  Điện thoại Smartphone (179)
, URL: https://tiki.vn/dien-thoai-smartphone/c1795?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1, ID: 20, Name:  Điện thoại bàn (102)
, URL: https://tiki.vn/dien-thoai-ban/c8061?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1, ID: 21, Name:  Điện thoại phổ thông (100)
, URL: https://tiki.vn/dien-thoai-pho-thong/c1796?src=c.1789.hamburger_menu_fly_out_banner, Parent: 1]
[]
[]
[]
[]
[]
[ID: 22, Name:  Máy giặt (696)
, URL: https://tiki.vn/may-giat/c3862?src=c.4221.hamburger_menu_fly_out_banner, Parent: 2, ID: 23, Name:  Máy lạnh - Máy điều hòa (651)
, URL: https://tiki.vn/may-lanh-may-dieu-hoa/c3865?src=c.4221.hamburger_menu_fly_out_banner, Parent: 2, ID: 24, Name:  Máy nước nón

ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))